
# 赛题介绍

百度网盘AI大赛——图像处理挑战赛是 百度网盘开放平台 面向AI开发者和爱好者发起的计算机视觉领域挑战赛。AI时代已到来，百度网盘与百度飞桨AI Studio强强联合，旨在基于个人云存储的生态能力开放，通过比赛机制，鼓励选手结合当下前沿的计算机视觉技术与图像处理技术，完成模型设计搭建与训练优化，产出基于飞桨框架的开源模型方案，为中国开源生态建设贡献一份力量。

本次图像处理挑战赛以线上比赛的形式进行，参赛选手需要在规定时间内，完成模型设计搭建与训练优化，并提交基于评测数据集产出的结果文件，榜单排名靠前且通过代码复查的队伍可获得高额奖金。

百度网盘开放平台 致力于为全球AI开发者和爱好者提供专业、高效的AI学习与开发环境，挖掘培养AI人才，助力技术产业生态发展。我们向所有胸怀梦想的开发者发出邀请，希望大家能与百度一起，用技术让复杂的世界变得更简单。

# 数据集简介

选手需要建立模型，对比赛给定的带有摩尔纹的图片进行处理，消除屏摄产生的摩尔纹噪声，还原图片原本的样子，并提交模型输出的结果图片。希望各位参赛选手结合当下前沿的计算机视觉技术与图像处理技术，在设计搭建模型的基础上，提升模型的训练性能、精度效果和泛化能力。在保证效果精准的同时，可以进一步考虑模型在实际应用中的性能表现，如更轻量、更高效等。

在本次比赛最新发布的数据集中，所有的图像数据均由真实场景采集得到，再通过技术手段进行相应处理，生成可用的脱敏数据集。该任务为image-to-image的形式，因此源数据和GT数据均以图片的形式来提供。各位选手可基于本次比赛最新发布的训练数据快速融入比赛，为达到更好的算法效果，本次比赛不限制大家使用额外的训练数据来优化模型。测试数据集的GT不做公开，请各位选手基于本次比赛最新发布的测试数据集提交对应的结果文件。

数据集构成
```
|- root  
    |- images
    |- gts
```
本次比赛最新发布的数据集共包含训练集、A榜测试集、B榜测试集三个部分，其中训练集共1000个样本，A榜测试集共200个样本，B榜测试集共200个样本；
images 为带摩尔纹的源图像数据，gts 为无摩尔纹的真值数据（仅有训练集数据提供gts ，A榜测试集、B榜测试集数据均不提供gts）；
images 与 gts 中的图片根据图片名称一一对应。

# 方案介绍

本方案基于基线调整了训练的Resize大小，从固定的一个值，变为不固定的的值，即输入的图片会被resize到一个并非同一的大小上。

一点点记录：

| Epoch | SSIM | PSNR | SCORE | MARKDOWN |
| -------- | -------- | -------- | -------- | -------- |
| 400| 0.96015| 28.85576 | 0.62436| 单张逐一训练共400epoch     |
| 320| 0.95375| 28.54452 | 0.6196| 单张逐一训练共400epoch     |
| \ | 	0.89157| 10.79841 | 0.49978	| \     |



In [ ]:
# 解压数据
%cd /home/aistudio/data/
!unzip /home/aistudio/data/data120844/moire_train_dataset.zip
!unzip /home/aistudio/data/data120844/moire_testA_dataset.zip

In [ ]:
# resize
%cd /home/aistudio/
import os
import cv2
for img_dir in os.listdir('data/moire_train_dataset/images'):
    if img_dir[-4:]=='.jpg':
        print(img_dir)
        img=cv2.imread('data/moire_train_dataset/images/'+img_dir)
        [w,h,c]=img.shape
        w=round(w/256)*256
        img=cv2.resize(img,(w,w))
        cv2.imwrite('data/moire_train_dataset/images/'+img_dir,img)
        img=cv2.imread('data/moire_train_dataset/gts/'+img_dir)
        img=cv2.resize(img,(w,w))
        cv2.imwrite('data/moire_train_dataset/gts/'+img_dir,img)

In [ ]:
# train
%cd /home/aistudio/demoire-baseline/\n
!python train_varsize.py --dataset_root ../data/moire_train_dataset/ --max_epochs 400 --batch_size 1 --sample_interval 400

In [4]:
# predict
%cd /home/aistudio/demoire-baseline/
!python predict.py --dataset_root ../data/moire_testA_dataset/ --pretrained train_result/model/epoch_320/model.pdparams 
%cd /home/aistudio/demoire-baseline/output/pre/
!zip result.zip *.jpg *.txt

/home/aistudio/demoire-baseline
W0112 06:42:18.295904   796 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0112 06:42:18.300663   796 device_context.cc:465] device: 0, cuDNN Version: 7.6.
Loading pretrained model from train_result/model/epoch_320/model.pdparams
There are 176/176 variables loaded into WDNet.
